### Sedaro API "Rolling Simulations" Example

This notebook shows a powerful use case of the Sedaro API where simulations are started and terminated on a rolling basis to inject incremental state corrections (i.e. from telemetry) over time.

This is particularly useful to operations use cases where telemetry can tune the simulation to match the real system(s) - enabling the simulation to predict future performance and fill telemetry gaps.

#### Important: Read Before Running

This notebook makes changes to agent and scenario branches indicated in the settings section. Ensure any changes to the target branches are saved prior to running this code. Sedaro recommends committing current work and creating new branches in the target repositories to avoid loss of work.

This notebook also requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
    "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.

In [ ]:
import json
from datetime import datetime
import time
from random import random

import requests
import matplotlib.pyplot as plt
from sedaro import SedaroApiClient, SedaroSimulationResult
from IPython import display
from tabulate import tabulate

In [ ]:
# Settings
with open('./secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']

with open('./config.json', 'r') as file:
    config = json.load(file)

# Obtain these IDs from the branch list within each repository and add to config.json
AGENT_TEMPLATE_BRANCH_ID = config['AGENT_TEMPLATE_BRANCH_ID']    # ID of the Wildfire vehicle template branch
SCENARIO_BRANCH_ID = config['SCENARIO_BRANCH_ID']                # ID of the Wildfire scenario template branch
HOST = config['HOST']                                            # Sedaro instance URL
WILDFIRE_AGENT_ID = 'NT06aqHUT5djI1_JPAsck'
WILDFIRE_FLIGHT_COMPUTER_ID = 'NT4VO8jS_HPyoLi2pu1tF'

In [ ]:
# Utils

def now_mjd():
    return datetime.utcnow().timestamp() / 86400 + 40587

def mjd_to_datetime(mjd):
    return datetime.utcfromtimestamp(round((mjd - 40587) * 86400))

#### Start a Simulation

Start a simulation from "now" for 2 hours into the future. 

This simulation is initialized using the latest TLE for the ISS from CelesTrak.  Before the simulation begins, Sedaro will propagate this TLE forward to the start time of the simulation.

In [ ]:
# Get latest TLEs from CelesTrak
response = requests.get('https://celestrak.org/NORAD/elements/gp.php?GROUP=stations&FORMAT=tle')
iss_tle = '\n'.join(response.text.split('ISS (ZARYA)')[1].split('\r\n')[1:3])
print('Using TLE:')
print(iss_tle)

In [ ]:
FUTURE_OFFSET = 60*2/60/24 # 2 hours

with SedaroApiClient(api_key=API_KEY, host=HOST) as sedaro_client:
    scenario = sedaro_client.get_branch(SCENARIO_BRANCH_ID)
    wildfire_agent_template = sedaro_client.get_branch(AGENT_TEMPLATE_BRANCH_ID)
    
    # Update Wildfire's orbit
    wildfire_agent = scenario.Agent.get(WILDFIRE_AGENT_ID)
    wildfire_agent.orbit.update(
        initialStateDefType='TLE',
        initialStateDefParams={ 'tle': iss_tle },
    )

    # Update simulation clock config
    start = now_mjd()
    stop = start + FUTURE_OFFSET
    clock = scenario.ClockConfig.get(scenario.data['clockConfig'])
    clock.update(
        startTime=start,
        stopTime=stop,
    )

    # Start simulation
    sim_client = sedaro_client.get_sim_client(SCENARIO_BRANCH_ID)
    simulation_job = sim_client.start()

print('Simulation started')

In [ ]:
wildfire_battery = wildfire_agent_template.Battery.get_first()
wildfire_flight_computer = wildfire_agent_template.Component.get(WILDFIRE_FLIGHT_COMPUTER_ID)

def fetch_results():
    def _fetch(tries=0):
        try:
            return SedaroSimulationResult.get_scenario_latest(API_KEY, SCENARIO_BRANCH_ID, host=HOST)
        except KeyboardInterrupt as e:
            raise e
        except:
            if tries < 5:
                print(f'No results yet.  Retrying after 10s... ({tries+1}/5)', end='\r')
                time.sleep(10)
                return _fetch(tries+1)
            raise ValueError(f'Failed to fetch results after {tries} tries')

    results = _fetch()

    wildfire_results = results.agent('Wildfire')
    root_results = wildfire_results.block('root')
    battery_results = wildfire_results.block(wildfire_battery.id)

    return wildfire_results, root_results, battery_results

wildfire_results, root_results, battery_results = fetch_results()
print('Done')

In [ ]:
def plot_results():
    display.clear_output(wait=True)
    
    common = {
        'linewidth': 1, 
        'elapsed_time': False, 
        'height': 2, 
        # 'xlim': [start, now], # CONFIG: Toggle this off to see all available past and future data
    }
    root_results.position.eci.plot(**common, ylabel='ECI Position', label=['X', 'Y', 'Z'])
    for pack in wildfire_battery.packs:
        wildfire_results.block(pack.id).temperature.degC.plot(
            **common, 
            show=False, 
            label=pack.name
        )
    wildfire_results.block(wildfire_flight_computer.id).temperature.degC.plot(
        **common, 
        ylabel='Temperatures [C]', 
        label=wildfire_flight_computer.name, 
        color='g'
    )
    battery_results.soc.plot(**common, ylabel='Battery SoC', color='m')
    battery_results.voltage.plot(**common, ylabel='Battery Voltage [V]', color='c')
    battery_results.current.plot(**common, ylabel='Battery Current [A]')

plot_results()

In [ ]:
def summarize_tlm(now, soc, v_batt, i_batt, t_fc, t_bpa, t_bpb, annotation):
    print(f'Telemetry @ {mjd_to_datetime(now)} UTC ({annotation}):')
    print(tabulate(
        [
            ('Battery State-of-Charge', soc, '%'),
            ('Battery Voltage', v_batt, 'V'),
            ('Battery Current', i_batt, 'A'),
            ('Flight Computer Temperature',  t_fc, 'deg. C'),
            ('Battery Pack A Temperature',  t_bpa, 'deg. C'),
            ('Battery Pack B Temperature',  t_bpb, 'deg. C'),
        ],
        ['Label', 'Value', 'Units'],
        tablefmt="github"
    ))

tlm = None
for i in range(120):

    now = now_mjd()

    # Fetch more data if necessary
    current_start = battery_results.soc.mjd[0]
    current_stop = battery_results.soc.mjd[-1]
    if (current_stop-now)/(current_stop-current_start) < 0.1:
        print('Fetching additional data...')
        wildfire_results, root_results, battery_results = fetch_results()

    sim_tlm = [
        battery_results.soc.value_at(now)*100, 
        battery_results.voltage.value_at(now),
        battery_results.current.value_at(now),
        wildfire_results.block(wildfire_flight_computer.id).temperature.degC.value_at(now),
        *[wildfire_results.block(pack.id).temperature.degC.value_at(now) for pack in wildfire_battery.packs]
    ]

    if i == 10:
        display.clear_output(wait=True)
        print('New telemetry received. Updating simulation...')

        # Inject real telemetry here
        # The following is a placeholder for real telemetry
        tlm = [t*(1-random()/30) for t in sim_tlm]
        tlm_timestamp = now

        # Compare telemetry to simulation
        errors = [abs((s-a)/a) for a, s in zip(tlm, sim_tlm)]

        # If the simulation is still running, first terminate it.
        try:
            response = sim_client.terminate(simulation_job['id'])
            print('Previous simulation terminated')
        except:
            print('Previous simulation already terminated')

        print('Starting new simulation with updated initial state...')

        with SedaroApiClient(api_key=API_KEY, host=HOST) as sedaro_client:
            scenario = sedaro_client.get_branch(SCENARIO_BRANCH_ID)
            
            # Update Wildfire's initial state to the prior simulation's state at the new telemetry's 
            # timestamp merged with all new telemetry
            wildfire_agent = scenario.Agent.get(WILDFIRE_AGENT_ID)
            todo

            # Update simulation clock config
            clock = scenario.ClockConfig.get(scenario.data['clockConfig'])
            clock.update(startTime=now, stopTime=now + FUTURE_OFFSET)

            # Start simulation
            sim_client = sedaro_client.get_sim_client(SCENARIO_BRANCH_ID)
            simulation_job = sim_client.start()

        print('Simulation started')

        wildfire_results, root_results, battery_results = fetch_results()

    display.clear_output(wait=True)

    if tlm:
        summarize_tlm(tlm_timestamp, *tlm, 'Actual')
        print('\nDeviation of simulated telemetry from actual telemetry:')
        print(f' - Max Error: {round(max(errors), 3)*100}%')
        print(f' - Min Error: {round(min(errors), 3)*100}%')
        print(f' - Average Error: {round(sum(errors)/len(errors), 3)*100}%')
        print('\n')
    summarize_tlm(now, *sim_tlm, 'Simulated')
    print(f'View Current Simulation: https://satellite.sedaro.com/#/scenario/{scenario.id}/edit/time')

    time.sleep(0.5)